### Install dependencies

In [1]:
from jupyter_dash import JupyterDash
import pandas as pd
import numpy as np
from plotly import graph_objects as go
import app as app_file
import setup as setup_file
import dash_core_components as dcc
import pickle
from configuration import Config

### Serve Jupyter Dash

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

### Define your static parameters

In [13]:
configs = Config()
setup_applet = setup_file.create_app(configs)
setup_applet.run_server(mode='inline')

### View your sample population

In [14]:
sample_pop = configs.params['enc_type'].initialize()
print(sample_pop)

[{'gene': array([29,  4, 11,  2,  5,  1, 24, 18, 19, 14, 10, 31,  7, 28,  8, 23, 25,
        21, 12, 26, 30,  0,  6, 22, 16,  9, 17, 15, 27,  3, 20, 13]), 'fitness': array([0]), 'meta': None}
 {'gene': array([ 6, 10, 24, 23, 17, 26, 18, 12, 15, 31,  7, 16,  5,  9, 22, 27,  1,
        28,  8, 30, 19, 13,  2,  4, 21, 20,  3, 14,  0, 11, 29, 25]), 'fitness': array([0]), 'meta': None}
 {'gene': array([ 0, 19, 27,  3,  8, 20, 15, 11, 16, 10, 24,  2, 22, 17, 26, 12,  1,
         6, 31,  4, 25, 30, 14, 23,  9,  7,  5, 18, 28, 29, 21, 13]), 'fitness': array([0]), 'meta': None}
 {'gene': array([24,  6, 18, 11,  7,  9, 25,  4, 22,  3, 27, 13, 16, 19,  8, 23, 20,
        21, 10, 12, 31, 26, 17, 28,  1,  5, 30,  2,  0, 29, 15, 14]), 'fitness': array([0]), 'meta': None}
 {'gene': array([ 1,  3, 21, 12, 11, 18, 25, 13, 22,  4, 28, 19, 27, 15, 16, 30, 20,
         0, 17, 24, 14,  7,  6,  9,  8, 10, 26, 31, 23,  2,  5, 29]), 'fitness': array([0]), 'meta': None}]


### Define your evaluation function

Your evaluation function should update all fitness values of an individual ```ind```. For example, the evaluation function for the single-objective Traveling Salesman Problem is defined below.

```
def eval(pop):
    dist = np.loadtxt('dist.txt')

    for ind in pop:
        ind['fitness'][0] = dist[0][ind['gene'][0]] + np.array([dist[ind['gene'][i - 1]][ind['gene'][i]] for i in range(1, configs.gene_size)]).sum() + dist[ind['gene'][-1]][0]

    return pop
```

In [15]:
def TSP(pop):
    dist = np.loadtxt('dist.txt')

    for ind in pop:
        ind['fitness'][0] = dist[0][ind['gene'][0]] + np.array([dist[ind['gene'][i - 1]][ind['gene'][i]] for i in range(1, configs.params['gene_size'])]).sum() + dist[ind['gene'][-1]][0]

    return pop

In [16]:
sample_ind = TSP(sample_pop)
print(sample_ind[0])

{'gene': array([29,  4, 11,  2,  5,  1, 24, 18, 19, 14, 10, 31,  7, 28,  8, 23, 25,
       21, 12, 26, 30,  0,  6, 22, 16,  9, 17, 15, 27,  3, 20, 13]), 'fitness': array([14753]), 'meta': None}


### Define a custom Plotly figure to display an individual's phenotypic (high-level) representation

For example, the phenotypic representation for a solution Traveling Salesman Problem is defined below as a graph network

```
def custom(ind):
    locs = np.loadtxt('loc.txt')  
    
    fig = go.Figure(data=[go.Scatter(x=[locs[0][0]] + [locs[0][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[0][0]],
                                     y=[locs[1][0]] + [locs[1][ind['gene'][i]] for i in range(configs.gene_size)] + [locs[1][0]],
                                     mode='lines+markers')])
    fig.update_layout(xaxis_title='Latitude', yaxis_title='Longitude')
    return fig
```

In [17]:
def network(ind):
    locs = np.loadtxt('loc.txt')  
    
    fig = go.Figure(data=[go.Scatter(x=[locs[0][0]] + [locs[0][ind['gene'][i]] for i in range(configs.params['gene_size'])] + [locs[0][0]],
                                     y=[locs[1][0]] + [locs[1][ind['gene'][i]] for i in range(configs.params['gene_size'])] + [locs[1][0]],
                                     mode='lines+markers')])
    fig.update_layout(xaxis_title='Latitude', yaxis_title='Longitude')
    return fig

### Save your static configurations

If you are satisfied with your evaluation function and custom visualization, save it to your configurations

In [18]:
configs.params['cust_eval'], configs.params['cust_vis'] = TSP, network

### Run the application

In [19]:
main_app = app_file.create_app(configs)
main_app.run_server()

Dash app running on http://127.0.0.1:8050/


In [20]:
from single import Single
configs.params['sel_type'] = Single(configs.params).tournament
configs.params['rep_type'] = Single(configs.params).tournament
configs.params['tourn_size'] = 8